In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import numpy as np
from sklearn.model_selection import train_test_split

No Bottleneck unit testing available.


In [2]:
stable = True # Set to True for latest pip version or False for main branch in GitHub
!pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [3]:
from tsai.all import *
import sklearn.metrics as skm
my_setup()

os              : Linux-5.10.0-11-amd64-x86_64-with-glibc2.31
python          : 3.9.2
tsai            : 0.3.9
fastai          : 2.7.15
fastcore        : 1.5.33
torch           : 2.2.2+cu121
device          : cpu
cpu cores       : 32
threads per cpu : 1
RAM             : 125.59 GB
GPU memory      : N/A


In [4]:
# dataset id
dsid ='NATOPS'
X, y, splits = get_UCR_data(dsid, return_split=False)
X.shape, y.shape, splits

((360, 24, 51),
 (360,),
 ((#180) [0,1,2,3,4,5,6,7,8,9...],
  (#180) [180,181,182,183,184,185,186,187,188,189...]))

In [5]:
set(y)

{'1.0', '2.0', '3.0', '4.0', '5.0', '6.0'}

In [6]:
target_values = []
indexes = []
for i in range(len(y)):
  if y[i] == '2.0':
    target_values.append(1)
    indexes.append(i)
  if y[i] == '3.0':
    target_values.append(2)
    indexes.append(i)
  if y[i] == '4.0':
     target_values.append(3)
     indexes.append(i)
  else:
    pass

data = X[indexes]

In [7]:
data = data[:,:6,:40]

In [8]:
data.shape


(180, 6, 40)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data,target_values,test_size=0.33, random_state=42)

In [10]:
# Node initialization

class Node:
    def __init__(self,feature=None, threshold=None,interval = None,relation =None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.interval = interval
        self.relation = relation
        self.left = left
        self.right = right
        self.value = value

# ALLEN RELATIONS       

def allen_relation(interval1, interval2):
    x, y = interval1
    a, b = interval2

    if y == a:
        return "A"  # after
    elif y < a:
        return "L"  # later
    elif x == a and b < y:
        return "B"  # begins
    elif y == b and x < a:
        return "E"  # ends
    elif x < a and b < y:
        return "D"  # during
    elif x < a < y < b:
        return "O"  # overlaps
    elif x == a and y < b:
        return "B_complement"  # begins in
    elif x > a and y < b:
        return "D_complement"  # during in
    elif a < x and b == y:
        return "E_complement"  # ends in
    elif a < x and x == b :
        return "A_complent"  # during begins
    elif a < b < x < y:
        return "L_complement"  # ends begins
    elif a< x < y < b:
        return "O_complement"  # ends overlaps
    else:
        return "No relation"

# CHECKING THRESHOLD 

def check_values_greater_than_threshold(lst, threshold, percentage):
    count_greater = sum(1 for value in lst if value > threshold)
    return count_greater / len(lst) >= percentage

def generate_all_intervals(t):
    intervals = []
    for start in range(t-4):  # Ensure the interval has 5 timestamps

        intervals.append((start, start+5))

    return intervals

#INFORMATION GAIN

def gini_impurity(y):
    _, counts = np.unique(y, return_counts=True)
    probs = counts / len(y)
    return - np.sum(probs*np.log(probs))

def information_gain(y, y_left, y_right):

    p = len(y_left) / len(y)
    gini_parent = gini_impurity(y)
    gini_children = p * gini_impurity(y_left) + (1 - p) * gini_impurity(y_right)
    return gini_parent - gini_children

# GENERATING TIME INTERVALS FOR TIME SERIES

def generate_all_intervals(t):
    intervals = []
    for start in range(t-7):  # Ensure the interval has 8 timestamps

        intervals.append((start, start+8))

    return intervals

# FINDING THE BEST DECISIONS FOR SPLITTING

def findBestDecision_and_split(data, target_values,poss_intervals=generate_all_intervals(30)):
    best_feature = None
    best_threshold = None
    best_interval = None
    max_information_gain = -1
    best_rel = None

    Te = None
    Tu = None
    left_target = None
    right_target = None


    for feature in range(data.shape[1]):
      unique_values_array = []
      for i in range(data.shape[0]):
          sample = data[i,feature, :]  # Extract the i-th sample
          unique_values = np.unique(sample)  # Find the unique  values for the current sample
          unique_values_array.append(unique_values)
          thresholds = []
          for row in unique_values_array:
              thresholds.extend(list(row))

      for j in range(len(possible_intervals)):

            ref_interval = possible_intervals[j]


            for relation in relations:

              after_intervals = [(a, b) for a, b in possible_intervals if allen_relation(ref_interval, (a, b)) == relation]


              for threshold in thresholds:

                    left_data = []
                    left_y = []
                    right_data = []
                    right_y = []


                    for i in range(len(data)):


                        int_values = [data[i][feature][interval[0]:interval[1]+1] for interval in after_intervals]


                        if any(check_values_greater_than_threshold(lst,threshold, 0.75) for lst in int_values):
                            left_data.append(data[i])
                            left_y.append(target_values[i])

                        else:
                            right_data.append(data[i])
                            right_y.append(target_values[i])


                    inf_gain = information_gain(target_values,left_y, right_y)


                    if inf_gain > max_information_gain:

                            best_interval = ref_interval
                            max_information_gain = inf_gain
                            best_threshold = threshold
                            best_rel = relation
                            best_feature = feature
                            Te = left_data
                            Tu = right_data
                            left_target = left_y
                            right_target = right_y


                        # print(len(left_target),len(right_target))

    return best_feature,best_interval,best_threshold,best_rel,np.array(Te),np.array(Tu),left_target,right_target,max_information_gain

In [11]:
def CreateLeafNode(target_values):
    if len(target_values) == 0:
        # Return a default value when target_values is empty
        return Node(value = None)
    else:
        # Return the most common target value
        return Node(value=np.argmax(np.bincount(list(target_values))))

def CreateNode(data):
    return Node()

def Learn(data,target_values,poss_intervals=generate_all_intervals(30),max_depth=8, current_depth=0):

    if len(target_values) == 0 :
        return CreateLeafNode(target_values)
    if len(np.unique(target_values)) == 1 or max_depth <= current_depth or poss_intervals ==[] or relations ==[] :
        return CreateLeafNode(target_values)


    best_feature,best_interval,best_threshold,best_rel,Te,Tu,left_target,right_target,max_information_gain = findBestDecision_and_split(data, target_values,poss_intervals=generate_all_intervals(30))

    node = CreateNode(data)
    node.feature = best_feature
    node.threshold = best_threshold
    node.interval = best_interval
    node.relation = best_rel

    print("Left tree shape : ",Te.shape)
    print("Target values in the left tree : ",left_target)
    print('*******************************')
    print("Right tree shape : ",Tu.shape)
    print("Target values in the right tree : ",right_target)
    print("Feature number used : ",best_feature,"\nBest Interval : ",best_interval,", Best relation : ",best_rel,", Best Threshold : ",best_threshold)
    print('__________________________________________________________________________')
    print()

    node.left = Learn(Te,left_target,poss_intervals = generate_all_intervals(30) ,max_depth = 8,current_depth = current_depth + 1)
    node.right = Learn(Tu,right_target,poss_intervals = generate_all_intervals(30),max_depth = 8 ,current_depth = current_depth+1)

    return node

def TCART(data,target_values,poss_intervals,relations):

    tau = Node()

    tau = Learn(data,target_values,poss_intervals,max_depth=8, current_depth=0)
    return tau


In [12]:

relations = ["A","L","B","E","D","O","B_complement","D_complement","E_complement","A_complent","L_complement","O_complement"]
possible_intervals = generate_all_intervals(40)

In [ ]:
tree  = TCART(X_train,y_train,poss_intervals = generate_all_intervals(40) ,relations=["A","L","B","E","D","O","B_complement","D_complement","E_complement","A_complent","L_complement","O_complement"])

Left tree shape :  (37, 6, 40)
Target values in the left tree :  [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
*******************************
Right tree shape :  (83, 6, 40)
Target values in the right tree :  [1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2]
Feature number used :  0 
Best Interval :  (0, 8) , Best relation :  L , Best Threshold :  -0.317034
__________________________________________________________________________



In [ ]:
def predict(node, sample):
    if node.value is not None:

        return node.value
    feature = node.feature
    threshold = node.threshold
    interval = node.interval
    relation = node.relation
    left = node.left
    right = node.right
   # value = node.value

    after_intervals = [(a, b) for a, b in possible_intervals if allen_relation(interval, (a, b)) == relation]
    int_values = [sample[feature][interval[0]:interval[1]+1] for interval in after_intervals]


    if any(check_values_greater_than_threshold(lst,threshold, 0.75) for lst in int_values):
        return predict(node.left, sample)
    else:
        return predict(node.right, sample)
 

predictions = []

# Iterate over each sample in the test data
for sample in X_test:
    # Predict the target variable for the sample using the decision tree 'tau'
    predicted_value = predict(tree,sample)
    predictions.append(predicted_value)

# Convert predictions to a numpy array
predictions = np.array(predictions)

# Print the predictions
print("Predictions:", predictions)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test,predictions)

print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

precision = precision_score(y_test, predictions, average= None)
recall = recall_score(y_test, predictions, average=None)
f1 = f1_score(y_test, predictions, average=None)

print("Precision :",precision)
print("Recall :", recall)
print("F1 Score :", f1)